In [ ]:
# !pip install SPARQLWrapper
import json, requests, xml
from xml.dom import minidom
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
from rdflib import Graph as RDFGraph
from rdflib.extras.external_graph_libs import rdflib_to_networkx_graph
import networkx as nx
from networkx import Graph as NXGraph
import matplotlib.pyplot as plt
import rdflib
import urllib
from matplotlib.pyplot import figure


In [ ]:
def load_dataframe( query_response ):
  processed_results = json.load(query_response)
  cols = processed_results['head']['vars']
  out = []
  for row in processed_results['results']['bindings']:
     item = []
     for c in cols:
         item.append(row.get(c, {}).get('value'))
     out.append(item)
  df = pd.DataFrame(out, columns=cols)
  return df


# FedEO API Tutorial - PART 3: SPARQL Online Demonstrator  
 This tutorial is part of a series of tutorials covering related subjects:                  
 * [FedEO API Tutorial - PART 1: OpenSearch and API Features](tutorial-1-api.ipynb)
 * [FedEO API Tutorial - PART 2: STAC](tutorial-2-stac.ipynb)
 * FedEO API Tutorial - PART 3: SPARQL Online
 * [FedEO API Tutorial - PART 4: SPARQL Offline](tutorial-4-sparql.ipynb)


<a name='Overview'></a>     
## Overview  
 The different parts covered in this Notebook are:    
 * [Collections](#Collections) 
 * [Services](#Services)
 * [Granules](#Granules)
       


<a name='Collections'></a>     
## Collections   


### Collections : Search by identifier  
**Example: 2**  
>  Return @id, name, instrument name, keywords.  
 OK  


In [ ]:
q="""

PREFIX schema: <https://schema.org/>

SELECT ?p ?name ?kw ?ins 
WHERE
{
  ?p schema:additionalType <http://purl.org/dc/dcmitype/Collection>.  
  ?p schema:identifier "TropForest".      
  ?p schema:name ?name.
  ?p schema:potentialAction/schema:instrument/schema:name ?ins.
  ?p schema:keywords ?kw. 
}
LIMIT 5

"""
sparql = SPARQLWrapper("https://eovoc.spacebel.be/sparql")
sparql.setQuery(q)
sparql.setReturnFormat(JSON)
result = sparql.query()
dataframe = load_dataframe(result.response)
dataframe.head(20)


### Collections : Search by keyword URI  
**Example: 3**  
>  Return @id, name, identifier, instrument name, keywords.  
 OK  


In [ ]:
q="""

PREFIX schema: <https://schema.org/>

SELECT ?p ?name ?identifier ?ins 
WHERE
{
  ?p schema:additionalType <http://purl.org/dc/dcmitype/Collection>.       
  ?p schema:name ?name.
  ?p schema:identifier ?identifier.
  ?p schema:potentialAction/schema:instrument/schema:name ?ins.
  ?p schema:keywords <https://earth.esa.int/concept/forestry>. 
}
LIMIT 5

"""
sparql = SPARQLWrapper("https://eovoc.spacebel.be/sparql")
sparql.setQuery(q)
sparql.setReturnFormat(JSON)
result = sparql.query()
dataframe = load_dataframe(result.response)
dataframe.head(20)


### Collections : Search by keyword, instrument  
**Example: 4**  
>  Return @id, name  
 OK  


In [ ]:
q="""

PREFIX schema: <https://schema.org/>

SELECT ?p ?name  
WHERE
{
  ?p schema:additionalType <http://purl.org/dc/dcmitype/Collection>.       
  ?p schema:name ?name.
  ?p schema:potentialAction/schema:instrument/schema:name "HRV".
  ?p schema:keywords "Agriculture" 
}
LIMIT 5

"""
sparql = SPARQLWrapper("https://eovoc.spacebel.be/sparql")
sparql.setQuery(q)
sparql.setReturnFormat(JSON)
result = sparql.query()
dataframe = load_dataframe(result.response)
dataframe.head(20)


### Collections : Distributed request finding collections matching INSPIRE Theme.  
**Example: 5**  
>  Without INSPIRE keywords being present in the catalogue.  
    


In [ ]:
q="""

PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX schema: <https://schema.org/>

SELECT ?entity ?name
WHERE
{
  
  ?entity  skos:inScheme <https://earth.esa.int/concepts/concept_scheme/earth-topics>.
  ?entity  skos:closeMatch <https://www.eionet.europa.eu/gemet/en/inspire-theme/lc>.
   
  SERVICE <https://eovoc.spacebel.be/sparql> {  

    ?p schema:additionalType <http://purl.org/dc/dcmitype/Collection>.       
    ?p schema:name ?name.
    ?p schema:keywords ?entity. 
      
  }

}
LIMIT 10
"""
sparql = SPARQLWrapper("http://eovoc.spacebel.be:8080/thesaurus/sparql")
sparql.setQuery(q)
sparql.setReturnFormat(JSON)
result = sparql.query()
dataframe = load_dataframe(result.response)
dataframe.head(20)


<a name='Services'></a>     
## Services   


### Services : Search by identifier  
**Example: 6**  
>  Return @id, name, keywords, ...  
 OK  


In [ ]:
q="""

PREFIX schema: <https://schema.org/>

SELECT ?p ?name ?kw   
WHERE
{
  ?p schema:additionalType <http://purl.org/dc/dcmitype/Service>.         
  ?p schema:name ?name.
  ?p schema:keywords ?kw. 
  ?p schema:identifier "rasdaman".
}
LIMIT 5

"""
sparql = SPARQLWrapper("https://eovoc.spacebel.be/sparql")
sparql.setQuery(q)
sparql.setReturnFormat(JSON)
result = sparql.query()
dataframe = load_dataframe(result.response)
dataframe.head(20)


### Services : Search by name  
**Example: 7**  
>  Return @id, identifier, keywords, ...  
 NOT OK - provider not returned.  


In [ ]:
q="""

PREFIX schema: <https://schema.org/>

SELECT ?p ?kw  ?id  ?provider 
WHERE
{
  ?p schema:additionalType <http://purl.org/dc/dcmitype/Service>.         
  ?p schema:name "SNAP".
  ?p schema:keywords ?kw. 
  ?p schema:identifier ?id.
  ?p schema:provider/schema:name ?provider.
}
LIMIT 5

"""
sparql = SPARQLWrapper("https://eovoc.spacebel.be/sparql")
sparql.setQuery(q)
sparql.setReturnFormat(JSON)
result = sparql.query()
dataframe = load_dataframe(result.response)
dataframe.head(20)


### Services : Search by provider  
**Example: 8**  
>  Return @id, identifier, keywords, ...  
 OK   


In [ ]:
q="""

PREFIX schema: <https://schema.org/>

SELECT ?p ?kw  ?id 
WHERE
{
  ?p schema:additionalType <http://purl.org/dc/dcmitype/Service>.         
  ?p schema:provider/schema:name "ESA/ESRIN".
  ?p schema:keywords ?kw. 
  ?p schema:identifier ?id.
}
LIMIT 5

"""
sparql = SPARQLWrapper("https://eovoc.spacebel.be/sparql")
sparql.setQuery(q)
sparql.setReturnFormat(JSON)
result = sparql.query()
dataframe = load_dataframe(result.response)
dataframe.head(20)


### Services : Search by provider URI  
**Example: 9**  
>  Return @id, identifier, keywords, ...  
 OK   


In [ ]:
q="""

PREFIX schema: <https://schema.org/>

SELECT ?p ?kw  ?id 
WHERE
{
  ?p schema:additionalType <http://purl.org/dc/dcmitype/Service>.         
  ?p schema:provider/schema:sameAs <https://dbpedia.org/resource/European_Space_Agency>.
  ?p schema:keywords ?kw. 
  ?p schema:identifier ?id.
}
LIMIT 5

"""
sparql = SPARQLWrapper("https://eovoc.spacebel.be/sparql")
sparql.setQuery(q)
sparql.setReturnFormat(JSON)
result = sparql.query()
dataframe = load_dataframe(result.response)
dataframe.head(20)


<a name='Granules'></a>     
## Granules   


### Granules : Search by platform name, Point of Interest URI  
**Example: 10**  
>  Return @id, identifier, keywords, polygon...  
 OK   


In [ ]:
q="""

PREFIX schema: <https://schema.org/>

SELECT ?p ?id ?polygon
WHERE
{
  ?p schema:additionalType <http://purl.org/dc/dcmitype/Dataset>.  
  ?p schema:potentialAction/schema:instrument/schema:name "PROBA".        
  ?p schema:spatialCoverage/schema:geoContains <http://yago-knowledge.org/resource/Mount_Etna>.
  ?p schema:spatialCoverage/schema:geo/schema:polygon ?polygon.
  ?p schema:identifier ?id.
}
LIMIT 5

"""
sparql = SPARQLWrapper("https://eovoc.spacebel.be/sparql")
sparql.setQuery(q)
sparql.setReturnFormat(JSON)
result = sparql.query()
dataframe = load_dataframe(result.response)
dataframe.head(20)


### Granules : Search by WKT geometry (coordinates of mount Etna)  
**Example: 11**  
>  Return @id, polygon, temporal coverage, thumbnail...  
**Example: 11**  
>  Use relation and mapping with schema.org as proposed in [GeoSPARQL 1.1](https://opengeospatial.github.io/ogc-geosparql/geosparql11/spec.html#_e_7_schema_org).  
 OK   


In [ ]:
q="""

PREFIX schema: <https://schema.org/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>

SELECT ?p ?id ?polygon ?time ?preview
WHERE
{
  ?p schema:additionalType <http://purl.org/dc/dcmitype/Dataset>.  
  ?p schema:potentialAction/schema:instrument/schema:name "PROBA".        
  ?p schema:spatialCoverage/schema:geoIntersects/geo:asWKT "POINT(14.995 37.755)"^^geo:wktLiteral.
  ?p schema:spatialCoverage/schema:geo/schema:polygon ?polygon.
  ?p schema:identifier ?id.
  ?p schema:temporalCoverage ?time.
  ?p schema:thumbnailUrl ?preview.
}
LIMIT 5

"""
sparql = SPARQLWrapper("https://eovoc.spacebel.be/sparql")
sparql.setQuery(q)
sparql.setReturnFormat(JSON)
result = sparql.query()
dataframe = load_dataframe(result.response)
dataframe.head(20)
